In [1]:
#사용 라이브러리
import os
import csv
from tqdm import tqdm
import numpy as np
import pandas as pd
from urllib.request import urlopen
from urllib import parse
from urllib.request import Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import json
pd.set_option('display.max_columns', 500)

In [2]:
candi = pd.read_csv('E:/post/logistics.csv', encoding='UTF-8', sep=",", error_bad_lines=False)

In [3]:
candi.head()

,상호명,성명,소재지,동수,일반창고,동수.1,냉동냉장,보관장소
0,(주)예인물류,한동호,인천광역시 중구 월미로 206 (북성동1가),0,0.00,0,0.0,"17,851.00"
1,주식회사 디와이물류,김대영,인천광역시 서구 백범로910번길 27 (가좌동),2,"3,958.00",0,0.0,"4,958.00"
2,씨제이대한통운(주),강신호,인천광역시 중구 서해대로209번길 13 (항동7가),0,0.00,0,0.0,"21,082.60"
3,(주)다해통상,김수정,인천광역시 중구 서해대로293번길 117 (항동7가),0,0,0,0.0,20888
4,씨제이대한통운㈜,강신호,"인천광역시 계양구 아나지로 529-1, 계양1SUB, 계양2SUB (서운동)",0,0,0,0.0,11588


In [4]:
#naver map api key
client_id = '';    # 본인이 할당받은 ID 입력
client_pw = '';    # 본인이 할당받은 Secret 입력

api_url = 'https://naveropenapi.apigw.gov-ntruss.com/map-geocode/v2/geocode?query='


# 네이버 지도 API 이용해서 위경도 찾기
geo_coordi = []
for add in tqdm(candi['소재지']):
    add_urlenc = parse.quote(add)  
    url = api_url + add_urlenc
    request = Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_pw)
    try:
        response = urlopen(request)
    except HTTPError as e:
        print('HTTP Error!')
        latitude = None
        longitude = None
    else:
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read().decode('utf-8')
            response_body = json.loads(response_body)   # json
            if response_body['addresses'] != []:
                longitude = response_body['addresses'][0]['x']
                latitude = response_body['addresses'][0]['y']
#                 print("Success!",end=' ')
            elif response_body['addresses'] == []:
                latitude = None
                longitude = None
#                 print("'result' not exist!",end=' ')
            else:
                print("'addresses' not exist!",end=' ')                
        else:
#             print('Response error code : %d' % rescode)
            latitude = None
            longitude = None

    geo_coordi.append([longitude,latitude])

100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [00:02<00:00,  7.59it/s]


In [5]:
np_geo_coordi = np.array(geo_coordi)
candi_geo = pd.DataFrame({"상호": candi['상호명'].values,
                              "소재지": candi['소재지'].values,
                              "경도": np_geo_coordi[:, 0],
                              "위도": np_geo_coordi[:, 1]})
candi_geo['GPS']=candi_geo['경도']+','+candi_geo['위도']

candi_geo.head()

,상호,소재지,경도,위도,GPS
0,(주)예인물류,인천광역시 중구 월미로 206 (북성동1가),126.5992272,37.4789776,"126.5992272,37.4789776"
1,주식회사 디와이물류,인천광역시 서구 백범로910번길 27 (가좌동),126.6574577,37.4895395,"126.6574577,37.4895395"
2,씨제이대한통운(주),인천광역시 중구 서해대로209번길 13 (항동7가),126.6218369,37.4462342,"126.6218369,37.4462342"
3,(주)다해통상,인천광역시 중구 서해대로293번길 117 (항동7가),126.6144167,37.4571513,"126.6144167,37.4571513"
4,씨제이대한통운㈜,"인천광역시 계양구 아나지로 529-1, 계양1SUB, 계양2SUB (서운동)",126.7517132,37.5265515,"126.7517132,37.5265515"


In [6]:
candi_geo.to_csv('E:/post/logistics_GPS.csv', mode='a', sep="|")

In [8]:
basecamp_geo=pd.DataFrame({'구분':['거점1','거점2'],
                          '소재지':['인천광역시 중구 서해대로 94번길 100','인천광역시 서구 원창동 391-28']})

In [9]:
#naver map api key
client_id = 'xhbfckkyg3';    # 본인이 할당받은 ID 입력
client_pw = 'lpd28lbrhk1WqcLPSmEzUUSAL0mrM5lwyMCAZoaz';    # 본인이 할당받은 Secret 입력

api_url = 'https://naveropenapi.apigw.gov-ntruss.com/map-geocode/v2/geocode?query='


# 네이버 지도 API 이용해서 위경도 찾기
geo_coordi = []
for add in tqdm(basecamp_geo['소재지']):
    add_urlenc = parse.quote(add)  
    url = api_url + add_urlenc
    request = Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_pw)
    try:
        response = urlopen(request)
    except HTTPError as e:
        print('HTTP Error!')
        latitude = None
        longitude = None
    else:
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read().decode('utf-8')
            response_body = json.loads(response_body)   # json
            if response_body['addresses'] != []:
                longitude = response_body['addresses'][0]['x']
                latitude = response_body['addresses'][0]['y']
#                 print("Success!",end=' ')
            elif response_body['addresses'] == []:
                latitude = None
                longitude = None
#                 print("'result' not exist!",end=' ')
            else:
                print("'addresses' not exist!",end=' ')                
        else:
#             print('Response error code : %d' % rescode)
            latitude = None
            longitude = None

    geo_coordi.append([longitude,latitude])
np_geo_coordi = np.array(geo_coordi)
basecamp_geo['경도'] = np_geo_coordi[:, 0]
basecamp_geo['위도'] = np_geo_coordi[:, 1]
basecamp_geo['GPS']=basecamp_geo['경도']+','+basecamp_geo['위도']

basecamp_geo.head()

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  8.32it/s]


,구분,소재지,경도,위도,GPS
0,거점1,인천광역시 중구 서해대로 94번길 100,126.6280371,37.4309900,"126.6280371,37.4309900"
1,거점2,인천광역시 서구 원창동 391-28,126.6327801,37.5089016,"126.6327801,37.5089016"


In [10]:
basecamp_geo.to_csv('E:/post/basecamp_geo.csv', mode='a', sep="|")